In [78]:
import pandas as pd
import numpy as np
import seaborn as sns

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [79]:
train = pd.read_csv("train.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [80]:
train["Embarked"].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [81]:
train = pd.read_csv("train.csv")

# Map female -> 0 and male -> 1
train["Sex"] = train["Sex"].map({"female": 0,
                                 "male": 1})

# In "Embarked" column there are {S:644, C:168, Q:77, NaN:2} so we map S -> 0, C -> 1, Q -> 2, NaN -> 0
embarked_map = {"S": 0,
              "C": 1,
              "Q": 2}

train["Embarked"] = train["Embarked"].map(embarked_map)
train.Embarked = train.Embarked.fillna(0)

# Remove categories that don't correspond to survival rate.
train = train.drop("PassengerId", axis=1)
train = train.drop("Name", axis=1)
train = train.drop("Ticket", axis=1)
train = train.drop("Cabin", axis=1)

# Put labels in their own dataframe
labels = train.pop("Survived")

train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,0.0
1,1,0,38.0,1,0,71.2833,1.0
2,3,0,26.0,0,0,7.9250,0.0
3,1,0,35.0,1,0,53.1000,0.0
4,3,1,35.0,0,0,8.0500,0.0


In [82]:
train.Age = (train.Age - train.Age.mean()) / train.Age.std()
train.Fare = (train.Fare - train.Fare.mean()) / train.Fare.std()
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,-0.530005,1,0,-0.502163,0.0
1,1,0,0.571430,1,0,0.786404,1.0
2,3,0,-0.254646,0,0,-0.488580,0.0
3,1,0,0.364911,1,0,0.420494,0.0
4,3,1,0.364911,0,0,-0.486064,0.0


In [83]:
# Replace null age values with a random number sampled from the continuous distribution [-1, +1] (this is mean +/- std of the ages)
null_ages = np.isnan(train.Age)
null_ages_count = len(null_ages)
random_ages = np.random.random_sample(null_ages_count)
random_ages = (random_ages - 0.5) * 2
train.Age[null_ages] = random_ages

/tmp/ipykernel_2640/3058668987.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  train.Age[null_ages] = random_ages
/tmp/ipykernel_2640/3058668987.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [84]:
# Set aside 200 samples for validation
train_val = train[:200]
labels_val = labels[:200]

# The remaining samples are for training
partial_train = train[200:]
partial_labels = labels[200:]

model = keras.Sequential([
    layers.Dense(4, activation="relu"),
    layers.Dense(4, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    partial_train,
    partial_labels,
    epochs=10,
    validation_data = (train_val, labels_val)
)

Epoch 1/10


2024-08-05 16:55:56.768413: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 16:55:56.920587: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 16:55:56.920654: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 16:55:56.924340: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-05 16:55:56.924408: I external/local_xla/xla/stream_executor

InternalError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/ben/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ben/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ben/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ben/.venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/ben/.venv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue

  File "/home/ben/.venv/lib/